In [5]:
import os
from dotenv import load_dotenv
import python_bithumb

# .env 파일 로드
load_dotenv()

# 환경 변수 가져오기
BITHUMB_API_KEY = os.getenv("BITHUMB_API_KEY")
BITHUMB_SECRET_KEY = os.getenv("BITHUMB_SECRET_KEY")

In [8]:
import time
import uuid
import jwt
import requests

# API URL 및 키 설정
api_url = "https://api.bithumb.com/v1/accounts"
access_key = BITHUMB_API_KEY # 본인의 Access Key로 교체
secret_key = BITHUMB_SECRET_KEY  # 본인의 Secret Key로 교체

# JWT payload 생성
payload = {
    'access_key': access_key,
    'nonce': str(uuid.uuid4()),
    'timestamp': str(int(time.time() * 1000))
}

# JWT 토큰 생성
jwt_token = jwt.encode(payload, secret_key, algorithm='HS256')
authorization_token = f'Bearer {jwt_token}'

# 요청 헤더 설정
headers = {
    'Authorization': authorization_token
}

# API 호출
try:
    response = requests.get(api_url, headers=headers)
    if response.status_code == 200:
        # JSON 응답 데이터
        data = response.json()
        # currency 값 호출
        for account in data:  # 응답이 여러 계좌 데이터를 포함할 수 있음
            print("Currency:", account.get('currency'))  # 각 계좌의 currency 출력
    else:
        print(f"Error: {response.status_code}, {response.text}")
except Exception as err:
    print(f"Exception occurred: {err}")


In [9]:
access_key = BITHUMB_API_KEY 
secret_key = BITHUMB_SECRET_KEY  

bithumb = python_bithumb.Bithumb(access_key, secret_key)

krw_balance = bithumb.get_balance("KRW")


print(krw_balance,"원")

10000.0 원


In [37]:
from datetime import datetime
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError
from python_bithumb.private_api import Bithumb
from python_bithumb.public_api import get_current_price

# Slack Bot Token과 채널 설정

SLACK_BOT_TOKEN = os.getenv("SLACK_BOT_TOKEN")
CHANNEL_NAME = os.getenv("CHANNEL_NAME")
client = WebClient(token=SLACK_BOT_TOKEN)

# 초기 자본과 API 초기화
INITIAL_CAPITAL = 1000000  # 예: 1,000,000원


def get_asset_summary():
    balances = bithumb.get_balances()
    total_balance = 0
    assets_info = []

    for asset in balances:
        currency = asset['currency']
        balance = float(asset['balance'])
        if balance > 0:
            current_price = get_current_price([f"KRW-{currency}"]) if currency != "KRW" else 1
            value = balance * current_price
            total_balance += value
            if currency != "KRW":
                assets_info.append(f"{currency}: {balance} (≈ {int(value)} KRW)")

    return total_balance, assets_info

def calculate_profit_rate(total_balance):
    return ((total_balance - INITIAL_CAPITAL) / INITIAL_CAPITAL) * 100

def send_slack_message(message):
    try:
        response = client.chat_postMessage(channel=SLACK_CHANNEL, text=message)
        assert response["ok"]
    except SlackApiError as e:
        print(f"Error sending message: {e.response['error']}")

def send_daily_report():
    total_balance, assets_info = get_asset_summary()
    profit_rate = calculate_profit_rate(total_balance)
    send_slack_message("정상적으로 프로그램이 동작중입니다.")
    
    if profit_rate <= -10:
        send_slack_message("★!★!★수익을 내는 코드를 수정하셔야 할 것 같습니다★!★!★!")
        print("프로그램 종료: 수익률 -10% 이하")
        return

    assets_info_str = "\n".join(assets_info)
    message = f"""
    --------------------------------------------------------------
    보유 자산 : {int(total_balance)}원
    보유 종목 :
    {assets_info_str}

    수익률 : {profit_rate:.2f}%
    --------------------------------------------------------------
    즐거운 연구생활 되세요!
    """
    send_slack_message(message)

if __name__ == "__main__":
    print("Daily Report 실행 중...")
    send_daily_report()


Daily Report 실행 중...


NameError: name 'SLACK_CHANNEL' is not defined

In [ ]:
import python_bithumb as bit

# 환경 변수 가져오기
BITHUMB_API_KEY = os.getenv("BITHUMB_API_KEY")
BITHUMB_SECRET_KEY = os.getenv("BITHUMB_SECRET_KEY")

access_key = BITHUMB_API_KEY 
secret_key = BITHUMB_SECRET_KEY  

bithumb = bit.Bithumb(access_key, secret_key)

krw_balance = bithumb.get_balance("KRW")

send_message(
    f"""
        안녕하세요 오전 8시마다 보내드리는 알림입니다.
    ------------------------------------
    전체 자산 : {krw_balance}원

    전체 수익률 : 

    전날 대비 수익률 : 
    ------------------------------------        
    활기찬 연구생활 되시기를 바랍니다.
    """, CHANNEL_NAME
)

import os
import pandas as pd
import python_bithumb

def save_scalping_ohlcv_to_csv(
    ticker: str = "KRW-BTC",
    interval: str = "minute1",
    count: int = 100,
    filename: str = "scalping_data.csv"
):
    """
    스캘핑을 위해 짧은 봉(예: 1분봉)의 OHLCV 데이터를 조회하여 CSV로 저장하는 함수 예시.
    
    Parameters
    ----------
    ticker : str
        'KRW-BTC', 'KRW-ETH', 'KRW-XRP' 등 Bithumb 시장 티커
    interval : str
        'minute1', 'minute3', 'minute5', 'minute30' 등
        python_bithumb가 지원하는 분봉 단위
    count : int
        몇 개의 봉 데이터를 가져올지 설정 (스캘핑 시 100~200 정도 사용 가능)
    filename : str
        저장할 CSV 파일 경로(이름). 예: "btc_scalping_data.csv"
    
    Returns
    -------
    None
        지정한 파일로 CSV 저장 후 함수 종료
    """
    # 1) 데이터 가져오기
    df = python_bithumb.get_ohlcv(ticker, interval=interval, count=count)
    if df is None or df.empty:
        print(f"[WARNING] {ticker} {interval} 데이터가 비어 있습니다. 저장을 중단합니다.")
        return
    
    # 2) CSV 파일 저장할 폴더가 없으면 생성
    dir_name = os.path.dirname(filename)
    if dir_name and not os.path.exists(dir_name):
        os.makedirs(dir_name, exist_ok=True)
    
    # 3) CSV로 저장 (한글 깨짐 방지 위해 utf-8-sig 사용)
    #    index=True 또는 False 선택 가능. 여기서는 True로 저장해 날짜 인덱스 포함
    df.to_csv(filename, encoding='utf-8-sig', index=True)
    
    print(f"[INFO] '{ticker}'의 {interval} 데이터 {count}개를 '{filename}'에 저장했습니다.")


if __name__ == "__main__":
    # 1분봉 기준, 최근 100개 봉 데이터를 "btc_scalping_data.csv"에 저장
    save_scalping_ohlcv_to_csv(
        ticker="KRW-BTC",
        interval="minute1",
        count=100,
        filename="btc_scalping_data.csv"
    )

    # 필요에 따라 3분봉, 5분봉 등도 따로 저장할 수 있음
    save_scalping_ohlcv_to_csv(
        ticker="KRW-BTC",
        interval="minute3",
        count=100,
        filename="btc_scalping_data_3min.csv"
    )
